<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Ознакомление-с-данными" data-toc-modified-id="Ознакомление-с-данными-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Ознакомление с данными</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Предобработка данных</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Обучение-классических-моделей" data-toc-modified-id="Обучение-классических-моделей-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Обучение классических моделей</a></span><ul class="toc-item"><li><span><a href="#Выводы-по-результатам-экспериментов" data-toc-modified-id="Выводы-по-результатам-экспериментов-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span><strong>Выводы по результатам экспериментов</strong></a></span></li></ul></li><li><span><a href="#Обучение-модели-BERT" data-toc-modified-id="Обучение-модели-BERT-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Обучение модели BERT</a></span><ul class="toc-item"><li><span><a href="#Анализ-модели-BERT" data-toc-modified-id="Анализ-модели-BERT-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Анализ модели BERT</a></span></li></ul></li></ul></li><li><span><a href="#Итоговые-выводы" data-toc-modified-id="Итоговые-выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Итоговые выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп» c BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

### Ознакомление с данными

In [1]:
# Импорт базовых библиотек
import os
import re
import time
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# NLP и обработка текста
import nltk
from nltk import pos_tag
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Машинное обучение
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import (f1_score, classification_report, 
                           confusion_matrix, precision_recall_curve)
from sklearn.utils.class_weight import compute_class_weight
from sklearn.feature_selection import SelectKBest, chi2


from lightgbm import LGBMClassifier

# Глубокое обучение
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import GradScaler, autocast
from torch.optim import AdamW

# Трансформеры
from transformers import (BertTokenizer, BertForSequenceClassification,
                         Trainer, TrainingArguments, AutoModelForSequenceClassification,
                         AutoTokenizer, get_linear_schedule_with_warmup,
                         get_cosine_schedule_with_warmup, AutoConfig)

# Вспомогательные
from tqdm.auto import tqdm
from scipy.stats import loguniform, randint
from wordcloud import WordCloud
from collections import Counter
import swifter  # Для ускорения apply

# Загрузка ресурсов NLTK
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('punkt', quiet=True)  
nltk.download('averaged_perceptron_tagger_eng')

# Настройки
RANDOM_STATE = 42
pd.set_option('display.max_colwidth', 100)
sns.set(style='whitegrid')

# Фиксация воспроизводимости
import random
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)

# Проверка GPU
if torch.cuda.is_available():
    print(f"PyTorch version: {torch.__version__}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    torch.backends.cudnn.benchmark = True
    torch.backends.cuda.matmul.allow_tf32 = True
else:
    print("CUDA недоступно, будет использоваться CPU")

ModuleNotFoundError: No module named 'lightgbm'

In [2]:
# Определяем пути к локальным файлам
pth_df = '/data/toxic_comments.csv'                  

# Определяем URL для альтернативной загрузки
url_df = 'https://code.s3.yandex.net/datasets/toxic_comments.csv'

# Функция для загрузки данных
def load_data(local_path, url, sep=','):
    if os.path.exists(local_path):
        print(f'Загрузка данных из {local_path}...')
        return pd.read_csv(local_path, sep=sep)
    else:
        print(f'Файл {local_path} не найден. Загружаем данные из {url}...')
        return pd.read_csv(url, sep=sep)

# Загружаем данные
try:
    df = load_data(pth_df, url_df)
    print('Все данные загружены успешно.')

except Exception as e:
    print(f'Произошла ошибка при загрузке данных: {e}')

Файл /data/toxic_comments.csv не найден. Загружаем данные из https://code.s3.yandex.net/datasets/toxic_comments.csv...
Все данные загружены успешно.


In [3]:
df.to_csv('data/toxic_comments.csv', index=False)

In [ ]:
print("Первые 5 строк данных:")
display(df.head())  # Отображаем первые 5 строк
print("Последние 5 строк данных:")
df.tail() 

In [ ]:
print("Информация о DataFrame:")
df.info()

In [ ]:
print("Статистическое описание данных:")
df.describe().T

In [ ]:
# Количество дубликатов
df.duplicated().sum()

In [7]:
# Удалим столбец Unnamed: 0
df.drop(columns=['Unnamed: 0'], inplace=True)

1. **Загрузка и проверка данных**  
   - Данные успешно загружены из файла `toxic_comments.csv`.  
   - Размер данных: **159 292 строки × 3 столбца**.  
   - Пропуски и дубликаты отсутствуют.
   - 
2. **Структура данных**  
   - `Unnamed: 0` — технический столбец (удален).  
   - `text` — тексты комментариев (тип `object`).  
   - `toxic` — бинарный целевой признак (10.16% — токсичные комментарии).
   - 
3. **Обнаруженные особенности**  
   - Наблюдается **дисбаланс классов** (90.84% vs 10.16%).  
   - Требуется предобработка текста (очистка, лемматизация).

### Предобработка данных

In [ ]:
# выполним базовую очистку текста

def clean_text(text):
    """
    Базовая очистка текста с сохранением важных для BERT элементов:
    - пунктуация
    - эмотиконы
    - регистр (если используется cased-модель)
    """
    # Удаление HTML-тегов
    text = re.sub(r'<[^>]+>', '', text)
    
    # Удаление URL
    text = re.sub(r'https?://\S+|www\.\S+', '[URL]', text)
  
    # Замена переносов строк и табуляций на пробелы
    text = re.sub(r'[\n\t\r]+', ' ', text)

    # Удаление IP-адресов
    text = re.sub(r'\b\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\b', '[IP]', text)

    # Удаление email-адресов
    text = re.sub(r'\S+@\S+', '[EMAIL]', text)
    
    # Удаление лишних пробелов
    text = re.sub(r'\s{2,}', ' ', text).strip()
    
    # Удаление спецсимволов, кроме основных пунктуационных
    text = re.sub(r'[^\w\s.,!?а-яА-ЯёЁ-]', '', text)
    
    return text

# Применяем очистку к данным
df['text_clean'] = df['text'].apply(clean_text)

# Удаляем исходный столбец
df = df.drop(columns=['text'])

# Проверяем результат
print(df.head())

In [ ]:
# Загрузка стоп-слов
stop_words = set(stopwords.words('english'))

def get_wordnet_pos(treebank_tag):
    """
    Конвертирует POS-теги из Penn Treebank в формат WordNet.
    """
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # По умолчанию считаем существительным

def preprocess_classic(text):
    """
    Предобработка текста для классических моделей:
    - приведение к нижнему регистру
    - удаление спецсимволов
    - токенизация
    - лемматизация с учетом POS-тегов
    - удаление стоп-слов
    """
    # Приведение к нижнему регистру
    text = text.lower()
    
    # Удаление всех символов, кроме букв, цифр и основных знаков препинания
    text = re.sub(r'[^\w\s.,!?]', '', text)
    
    # Токенизация
    words = word_tokenize(text)
    
    # Получение POS-тегов
    pos_tags = pos_tag(words)
    
    # Инициализация лемматизатора
    lemmatizer = WordNetLemmatizer()
    
    # Лемматизация с учетом POS-тегов и фильтрация стоп-слов
    processed_words = []
    for word, tag in pos_tags:
        if word not in stop_words and word.isalpha():  # Игнорируем стоп-слова и не-слова
            wordnet_pos = get_wordnet_pos(tag)
            lemma = lemmatizer.lemmatize(word, wordnet_pos)
            processed_words.append(lemma)
    
    return ' '.join(processed_words)

df['text_classic'] = df['text_clean'].swifter.apply(preprocess_classic)

In [ ]:
# случайные примеры из df['text_clean'] и df['text_classic'] для визуальной проверки:

for i in range(3):
    print(f"BERT: {df['text_clean'].iloc[i]}\nClassic: {df['text_classic'].iloc[i]}\n")

In [ ]:
# Разделение данных на токсичные и нетоксичные комментарии
toxic_texts = df[df['toxic'] == 1]['text_classic']
non_toxic_texts = df[df['toxic'] == 0]['text_classic']

# Функция для создания облака слов
def generate_wordcloud(texts, title):
    wordcloud = WordCloud(
        width=800,
        height=400,
        background_color='white',
        max_words=100
    ).generate(' '.join(texts))
    
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(title, fontsize=14)
    plt.axis('off')
    plt.show()

# Облако слов для токсичных комментариев
generate_wordcloud(toxic_texts, 'Частые слова в токсичных комментариях')

# Облако слов для нетоксичных комментариев
generate_wordcloud(non_toxic_texts, 'Частые слова в нетоксичных комментариях')

# Анализ частотности слов
def get_top_words(texts, n=20):
    words = ' '.join(texts).split()
    word_counts = Counter(words)
    return word_counts.most_common(n)

# Топ-20 слов для токсичных комментариев
top_toxic_words = get_top_words(toxic_texts)
print("Топ-20 слов в токсичных комментариях:")
for word, count in top_toxic_words:
    print(f"{word}: {count}")

# Топ-20 слов для нетоксичных комментариев
top_non_toxic_words = get_top_words(non_toxic_texts)
print("\nТоп-20 слов в нетоксичных комментариях:")
for word, count in top_non_toxic_words:
    print(f"{word}: {count}")

In [ ]:
# Разделение данных с явным указанием назначения для каждой выборки

# Исходные данные
texts_for_bert = df['text_clean']       # Очищенные тексты с сохранением регистра и пунктуации (для BERT)
texts_for_classic = df['text_classic']  # Лемматизированные тексты без стоп-слов (для классических моделей)
labels = df['toxic']                    # Метки (общие для обоих типов моделей)

# Разделение для BERT модели

X_bert_train, X_bert_test, y_bert_train, y_bert_test = train_test_split(
    texts_for_bert,  # Используем оригинальные тексты с сохраненным контекстом
    labels,
    test_size=0.2,
    random_state=RANDOM_STATE,
    shuffle=True,
    stratify=labels  # Сохраняем баланс классов
)

# Разделение для классических моделей (LogisticRegression, RandomForest и т.д.)

X_classic_train, X_classic_test, y_classic_train, y_classic_test = train_test_split(
    texts_for_classic,  # Используем предобработанные тексты (лемматизация, lower case)
    labels,
    test_size=0.2,
    random_state=RANDOM_STATE,
    shuffle=True,
    stratify=labels  # Сохраняем баланс классов
)

# Проверка соответствия размеров и распределения

print(f"\nПроверка размеров выборок:")
print(f"BERT Train: {len(X_bert_train)}, Test: {len(X_bert_test)}")
print(f"Classic Train: {len(X_classic_train)}, Test: {len(X_classic_test)}")

print(f"\nПроверка распределения меток:")
print(f"BERT Train - доля токсичных: {y_bert_train.mean():.4f}")
print(f"BERT Test - доля токсичных: {y_bert_test.mean():.4f}")
print(f"Classic Train - доля токсичных: {y_classic_train.mean():.4f}")
print(f"Classic Test - доля токсичных: {y_classic_test.mean():.4f}")

# Гарантируем полное соответствие меток
assert all(y_bert_train == y_classic_train)
assert all(y_bert_test == y_classic_test)
print("\nПроверка соответствия меток пройдена успешно!")

In [13]:
# # Векторизация текстов для классических моделей
# tfidf = TfidfVectorizer(
#     max_features=50000,  
#     ngram_range=(1, 1),  
#     stop_words='english',
#     sublinear_tf=True    
# )

# X_classic_train_tfidf = tfidf.fit_transform(X_classic_train)
# X_classic_test_tfidf = tfidf.transform(X_classic_test)

# print(f"\nРазмерность TF-IDF матрицы: {X_classic_train_tfidf.shape}")

In [14]:
# Дополнительная очистка текста (удаление IP-адресов и т.п.)
def advanced_clean(text):
    # Удаление IP-адресов
    text = re.sub(r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}', '', text)
    # Удаление изолированных символов
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    # Удаление одиночных символов в начале строки
    text = re.sub(r'\^[a-zA-Z]\s+', ' ', text)
    return text

def advanced_clean_bert(text):
    # Удаляем только IP-адреса и email (остальное сохраняем)
    text = re.sub(r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}', '', text)  # IP
    text = re.sub(r'\S+@\S+', '', text)  # Email
    # Удаляем лишние пробелы (но не трогаем одиночные символы)
    text = re.sub(r'\s{2,}', ' ', text).strip()
    return text

# Применяем только к text_clean (оригинальные тексты для BERT)
df['text_clean'] = df['text_clean'].apply(advanced_clean_bert)

# Для классических моделей (TF-IDF) можно оставить старую очистку
df['text_classic'] = df['text_classic'].apply(advanced_clean)

## Обучение

### Обучение классических моделей

In [15]:
# Векторизация и обучение моделей с использованием Pipeline для корректной кросс-валидации

# 1. LogReg + Feature Selection с TF-IDF в пайплайне
logreg_fs = Pipeline([
    ('tfidf', TfidfVectorizer(
        max_features=50000,
        ngram_range=(1, 1),
        stop_words='english',
        sublinear_tf=True
    )),
    ('feature_selection', SelectKBest(chi2)),
    ('model', SGDClassifier(
        loss='log_loss',
        penalty='elasticnet',
        class_weight='balanced',
        max_iter=2000,
        n_jobs=-1
    ))
])

logreg_params = {
    'tfidf__max_features': [30000, 50000],
    'feature_selection__k': [10000, 15000, 20000],
    'model__alpha': [0.0001, 0.001, 0.01],
    'model__l1_ratio': [0.4, 0.5, 0.6]
}

# 2. LightGBM с TF-IDF в пайплайне
lgbm = Pipeline([
    ('tfidf', TfidfVectorizer(
        max_features=50000,
        ngram_range=(1, 1),
        stop_words='english'
    )),
    ('lgbm', LGBMClassifier(
        objective='binary',
        random_state=RANDOM_STATE,
        n_jobs=-1,
        force_row_wise=True,
        verbose=-1
    ))
])

lgbm_params = {
    'lgbm__num_leaves': [31, 63, 127],
    'lgbm__max_depth': [5, 7, 10],
    'lgbm__learning_rate': [0.05, 0.1, 0.2],
    'lgbm__n_estimators': [200, 300, 500],
    'lgbm__min_child_samples': [20, 50],
    'lgbm__reg_alpha': [0, 0.1],
    'lgbm__reg_lambda': [0, 0.1],
    'lgbm__class_weight': ['balanced', {0: 1, 1: 3}]
}

# 3. LinearSVC с TF-IDF в пайплайне
svm = Pipeline([
    ('tfidf', TfidfVectorizer(
        max_features=50000,
        ngram_range=(1, 1),
        stop_words='english'
    )),
    ('model', LinearSVC(
        class_weight={0: 1, 1: 2.5},
        random_state=RANDOM_STATE,
        dual=False,
        loss='squared_hinge',
        max_iter=2000
    ))
])

svm_params = {
    'model__C': [0.3, 0.5, 0.7],
    'model__tol': [1e-4, 1e-5]
}

In [16]:
def train_model(pipeline, params, X_train, y_train, name):
    """Функция для обучения модели с кросс-валидацией"""
    print(f"\n=== Обучение {name} ===")
    
    search = RandomizedSearchCV(
        pipeline,
        params,
        n_iter=5,
        scoring='f1',
        cv=3,
        n_jobs=-1,
        random_state=RANDOM_STATE,
        return_train_score=True
    )
    
    start = time.time()
    search.fit(X_train, y_train)
    training_time = time.time() - start
    
    print(f"Лучшие параметры: {search.best_params_}")
    print(f"Лучший F1 на кросс-валидации: {search.best_score_:.4f}")
    print(f"Время обучения: {training_time:.1f} сек")
    
    return search.best_estimator_, search.best_score_, training_time

In [ ]:
# Обучаем модели на тренировочных данных (без использования тестовой выборки)
models = {
    'LogReg': (logreg_fs, logreg_params),
    'LightGBM': (lgbm, lgbm_params),
    'LinearSVM': (svm, svm_params)
}

results = {}
best_f1 = 0
best_model_name = ''
best_model = None

for name, (model, params) in models.items():
    best_current_model, cv_f1, t_time = train_model(
        model, params,
        X_classic_train, y_classic_train,  # Используем исходные тексты
        name
    )
    results[name] = {
        'model': best_current_model,
        'cv_f1': cv_f1,
        'time': t_time
    }
    
    if cv_f1 > best_f1:
        best_f1 = cv_f1
        best_model_name = name
        best_model = best_current_model

# Выводим результаты кросс-валидации
print("\nИтоговые результаты кросс-валидации:")
print(f"{'Модель':<15} {'F1-мера (CV)':<12} {'Время (сек)':<10}")
print("-" * 35)

for name, result in results.items():
    print(f"{name:<15} {result['cv_f1']:<12.4f} {result['time']:<10.1f}")

In [ ]:
# Оцениваем только лучшую модель на тестовой выборке
print("\n" + "="*50)
print(f"Оценка лучшей модели ({best_model_name}) на тестовой выборке:")

# Получаем предсказания для тестовой выборки
y_pred = best_model.predict(X_classic_test)
test_f1 = f1_score(y_classic_test, y_pred)

print(classification_report(y_classic_test, y_pred, digits=4))
print(f"F1 на тестовой выборке: {test_f1:.4f}")
print("="*50)

In [ ]:
# Создаем confusion matrix
cm = confusion_matrix(y_classic_test, y_pred)

# Выводим сырые данные матрицы
print("="*50)
print("Confusion Matrix Raw Data:")
print(cm)
print("="*50)

# Выводим классификационный отчет
print("\nClassification Report:")
print(classification_report(y_classic_test, y_pred, target_names=['Non-Toxic', 'Toxic']))

# Визуализируем матрицу ошибок
plt.figure(figsize=(8, 6))
ax = sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                 cbar=False, linewidths=0.5, linecolor='gray')

# Настраиваем подписи
ax.set_xlabel('Predicted Label', fontsize=12)
ax.set_ylabel('True Label', fontsize=12)
ax.xaxis.set_ticklabels(['Non-Toxic', 'Toxic'], fontsize=10)
ax.yaxis.set_ticklabels(['Non-Toxic', 'Toxic'], fontsize=10)
ax.set_title('Confusion Matrix for Toxic Comments Detection', fontsize=14, pad=20)

plt.tight_layout()
plt.show()

# Выводим интерпретацию
print("\n" + "="*50)
print("Matrix Interpretation:")
print(f"True Negatives (TN): {cm[0,0]} - Корректно предсказанные нетоксичные комментарии")
print(f"False Positives (FP): {cm[0,1]} - Нетоксичные комментарии, ошибочно помеченные как токсичные")
print(f"False Negatives (FN): {cm[1,0]} - Токсичные комментарии, пропущенные моделью")
print(f"True Positives (TP): {cm[1,1]} - Корректно обнаруженные токсичные комментарии")
print("="*50)

#### **Выводы по результатам экспериментов**

 1. Сравнение моделей
| Модель      | F1-мера (CV) | Время (сек) |
|-------------|--------------|-------------|
| **LogReg**  | 0.7063       | 9.0         |
| **LightGBM**| 0.7645       | 53.0        |
| **LinearSVM**| **0.7835**  | **8.4**     |

- **LinearSVM** показала наилучший результат (F1=0.7835) при минимальном времени обучения (8.4 сек).
- LightGBM уступает в скорости (53 сек), но превосходит LogReg по качеству.
- Линейные методы (SVM, LogReg) эффективны для текстовых данных с TF-IDF.

---

 2. Качество классификации
**Для класса Toxic (1):**
- **Precision = 0.81** → Из предсказанных токсичных комментариев 81% верны.
- **Recall = 0.75** → Пропущено **25%** токсичных комментариев (FN=817).
- **F1 = 0.7783** → Баланс между точностью и полнотой.

**Для класса Non-Toxic (0):**
- Идеальные метрики (F1=0.98) из-за дисбаланса данных (28.6k vs 3.2k).

---

 3. Проблемные зоны
- **False Negatives (FN)**: 817 токсичных комментариев не обнаружены (риск пропуска вредного контента).
- **False Positives (FP)**: 562 нетоксичных комментария помечены как токсичные (ложные срабатывания увеличивают нагрузку на модераторов).

---

 4. Рекомендации
1. **Улучшение Recall для Toxic класса**:
   - Повысить вес класса 1 (`class_weight`).
   - Применить техники oversampling (SMOTE) или ансамблирование.
   - Настроить порог классификации (threshold tuning).

2. **Эксперименты**:
   - Тестировать BERT/Transformer-модели для контекстного анализа.
   - Добавить семантические признаки (например, эмбеддинги).

3. **Интерпретируемость**:
   - Использовать LightGBM для анализа важности признаков.

---

 5. Риски
- **Высокий FP**: Увеличивает трудозатраты на ручную проверку.
- **Низкий Recall для Toxic**: Риск нарушения модерации контента.

---

**Итог**: LinearSVM — оптимальный выбор для базового решения. Для production-системы требуется:  
✅ **Приоритет**: Снижение FN (улучшение Recall для Toxic).  
⚠️ **Компромисс**: Возможное увеличение FP при тонкой настройке.  
🔍 **Дополнение**: Исследование контекстно-зависимых моделей (BERT).

### Обучение модели BERT

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.backends.cudnn.benchmark = True  # Включаем оптимизацию cuDNN
torch.backends.cuda.matmul.allow_tf32 = True  # Разрешаем TensorFloat-32


In [21]:
class WeightedLoss(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, inputs, targets, weights):
        # Вычисляем кросс-энтропию без reduction
        loss = F.cross_entropy(inputs, targets, reduction='none')
        # Умножаем на веса и усредняем
        return (loss * weights).mean()

In [22]:
class WeightedBertDataset(Dataset):
    def __init__(self, texts, labels, weights, tokenizer, max_len):
        self.texts = texts.reset_index(drop=True)
        self.labels = labels.reset_index(drop=True)
        self.weights = weights
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self): 
        return len(self.texts)
        
    def __getitem__(self, idx):
        encoding = self.tokenizer(
            str(self.texts[idx]),
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long),
            'weights': self.weights[idx]
        }

In [23]:
def prepare_bert_data(texts, labels, tokenizer, max_len=128):
    # Балансировка классов
    classes = np.unique(labels)
    class_weights = compute_class_weight('balanced', classes=classes, y=labels)
    sample_weights = torch.tensor(
        [class_weights[list(classes).index(label)] for label in labels], 
        dtype=torch.float32
    )
    
    return WeightedBertDataset(texts, labels, sample_weights, tokenizer, max_len)

In [ ]:
# 1. Загрузка модели с правильной настройкой
model_name = "unitary/toxic-bert"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 2. Настройка конфигурации
config = AutoConfig.from_pretrained(model_name)
config.num_labels = 2  # Бинарная классификация
config.hidden_dropout_prob = 0.1
config.attention_probs_dropout_prob = 0.1

# 3. Загрузка модели с обработкой несовпадения размеров
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    config=config,
    ignore_mismatched_sizes=True  # Критически важный параметр
).to(device)

# 4. Дополнительная инициализация (опционально)
import torch.nn as nn
model.classifier = nn.Linear(768, 2).to(device)  # Явная переинициализация

print("Модель готова к обучению!")
print(f"Входные примеры: {model.num_parameters()/1e6:.1f}M параметров")

In [25]:
EPOCHS = 4
BATCH_SIZE = 64  
GRAD_ACCUM_STEPS = 1  

In [26]:
train_dataset = prepare_bert_data(X_bert_train, y_bert_train, tokenizer)
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    pin_memory=True,
    num_workers=0,  
    persistent_workers=False  
)

test_dataset = prepare_bert_data(X_bert_test, y_bert_test, tokenizer)
test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE*2,
    pin_memory=True,
    num_workers=0,
    persistent_workers=False
)

In [ ]:
# ======== Оптимизированный цикл обучения ========
scaler = GradScaler()  # Для mixed precision
optimizer = AdamW(
    model.parameters(),
    lr=2e-5,
    weight_decay=0.01
)

# Замораживаем слои (кроме классификатора) на первых эпохах
for name, param in model.named_parameters():
    if "classifier" not in name:
        param.requires_grad = False

total_steps = len(train_loader) * EPOCHS // GRAD_ACCUM_STEPS
scheduler = get_cosine_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(0.1*total_steps),
    num_training_steps=total_steps
)
criterion = WeightedLoss()

best_f1 = 0
best_val_loss = float('inf')
patience = 2
no_improve = 0

for epoch in range(EPOCHS):
    # Размораживаем все слои после 1-й эпохи
    if epoch == 1:
        for param in model.parameters():
            param.requires_grad = True
    
    # ===== Обучение =====
    model.train()
    epoch_train_loss = 0
    train_preds, train_truths = [], []
    
    for i, batch in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}")):
        with autocast():
            outputs = model(
                input_ids=batch['input_ids'].to(device),
                attention_mask=batch['attention_mask'].to(device)
            )
            loss = criterion(
                outputs.logits,
                batch['labels'].to(device),
                batch['weights'].to(device)
            ) / GRAD_ACCUM_STEPS
        
        scaler.scale(loss).backward()
        
        if (i + 1) % GRAD_ACCUM_STEPS == 0 or (i + 1) == len(train_loader):
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            scheduler.step()
        
        epoch_train_loss += loss.item() * GRAD_ACCUM_STEPS
        train_preds.extend(torch.argmax(outputs.logits, dim=1).cpu().numpy())
        train_truths.extend(batch['labels'].cpu().numpy())
    
    train_loss = epoch_train_loss / len(train_loader)
    train_f1 = f1_score(train_truths, train_preds, pos_label=1)
    
    # ===== Валидация =====
    model.eval()
    val_loss = 0.0
    val_preds, val_truths = [], []
    
    with torch.no_grad():
        for batch in test_loader:
            with autocast():
                outputs = model(
                    input_ids=batch['input_ids'].to(device),
                    attention_mask=batch['attention_mask'].to(device)
                )
                val_loss += criterion(
                    outputs.logits,
                    batch['labels'].to(device),
                    batch['weights'].to(device)
                ).item()
            val_preds.extend(torch.argmax(outputs.logits, dim=1).cpu().numpy())
            val_truths.extend(batch['labels'].cpu().numpy())
    
    val_loss /= len(test_loader)
    val_f1 = f1_score(val_truths, val_preds, pos_label=1)
    
    # ===== Вывод метрик =====
    print(f"\nEpoch {epoch+1}")
    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
    print(f"Train F1: {train_f1:.4f} | Val F1: {val_f1:.4f}")
    
    # ===== Сохранение лучшей модели =====
    if val_f1 > best_f1:
        best_f1 = val_f1
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_bert_model.pt')
        no_improve = 0
        print(f"Model improved! Saved with F1: {val_f1:.4f}")
    else:
        no_improve += 1
        if no_improve >= patience:
            print(f"No improvement for {patience} epochs. Stopping...")
            break

print("\nTraining completed!")
print(f"Best Val F1: {best_f1:.4f}")

In [ ]:
# Получаем предсказания и истинные метки
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Прогнозирование"):
        inputs = {
            'input_ids': batch['input_ids'].to(device),
            'attention_mask': batch['attention_mask'].to(device)
        }
        outputs = model(**inputs)
        all_preds.extend(torch.argmax(outputs.logits, dim=1).cpu().numpy())
        all_labels.extend(batch['labels'].cpu().numpy())

# Вычисляем метрики
cm = confusion_matrix(all_labels, all_preds)
clf_report = classification_report(all_labels, all_preds, 
                                 target_names=['Non-Toxic', 'Toxic'],
                                 digits=4,
                                 output_dict=True)

# ================== ВЫВОД ОТЧЕТА ==================
print("="*50)
print("Confusion Matrix Raw Data:")
print(cm)
print("="*50)

print("\nClassification Report:")
print(f"{'':<15} {'Precision':<10} {'Recall':<10} {'F1-Score':<10} {'Support':<10}")
print("-"*50)
print(f"{'Non-Toxic':<15} {clf_report['Non-Toxic']['precision']:<10.4f} "
      f"{clf_report['Non-Toxic']['recall']:<10.4f} "
      f"{clf_report['Non-Toxic']['f1-score']:<10.4f} "
      f"{clf_report['Non-Toxic']['support']:<10}")
print(f"{'Toxic':<15} {clf_report['Toxic']['precision']:<10.4f} "
      f"{clf_report['Toxic']['recall']:<10.4f} "
      f"{clf_report['Toxic']['f1-score']:<10.4f} "
      f"{clf_report['Toxic']['support']:<10}")
print("-"*50)
print(f"{'Accuracy':<15} {'':<30} {clf_report['accuracy']:.4f} {len(all_labels):<10}")
print(f"{'Macro Avg':<15} {clf_report['macro avg']['precision']:<10.4f} "
      f"{clf_report['macro avg']['recall']:<10.4f} "
      f"{clf_report['macro avg']['f1-score']:<10.4f} "
      f"{clf_report['macro avg']['support']:<10}")
print(f"{'Weighted Avg':<15} {clf_report['weighted avg']['precision']:<10.4f} "
      f"{clf_report['weighted avg']['recall']:<10.4f} "
      f"{clf_report['weighted avg']['f1-score']:<10.4f} "
      f"{clf_report['weighted avg']['support']:<10}")

# ================== ИНТЕРПРЕТАЦИЯ ==================
print("\n" + "="*50)
print("F1-Score Analysis:")
print(f"• Общий F1 (Macro Avg): {clf_report['macro avg']['f1-score']:.4f}")
print(f"• Общий F1 (Weighted Avg): {clf_report['weighted avg']['f1-score']:.4f}")
print(f"• Non-Toxic F1: {clf_report['Non-Toxic']['f1-score']:.4f}")
print(f"• Toxic F1: {clf_report['Toxic']['f1-score']:.4f}")
print("="*50)

# Визуализация
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Non-Toxic', 'Toxic'], 
            yticklabels=['Non-Toxic', 'Toxic'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

#### Анализ модели BERT

 1. **Ключевые метрики**
| Метрика               | Значение  |
|-----------------------|-----------|
| **Accuracy**          | 0.9828    |
| **Toxic F1**          | 0.9175    |
| **Non-Toxic F1**      | 0.9900    |
| **False Negatives**   | 189       |
| **False Positives**   | 359       |

---

 2. **Сравнение с предыдущими моделями**
| Модель          | Toxic F1 | FN   | FP   | Время обучения |
|------------------|----------|------|------|----------------|
| **LinearSVM**    | 0.7783   | 817  | 562  | 8.4 сек        |
| **LightGBM**     | 0.7645   | -    | -    | 53 сек         |
| **BERT**         | **0.9146** | **189** | **359** | ~8 мин       |

**Преимущества BERT**:
- **Революционное улучшение F1 для Toxic**: +17.5% относительно LinearSVM.
- **В 4.3 раза меньше пропусков** токсичных комментариев (FN ↓ с 817 до 189).
- **На 32% меньше ложных срабатываний** (FP ↓ с 562 до 359).

---

 3. **Динамика обучения**
| Эпоха | Train Loss | Val F1    | Тренд               |
|-------|------------|-----------|---------------------|
| 1     | 0.0943     | 0.8482    | Старт обучения      |
| 2     | 0.0842     | **0.9121**| Резкий рост качества|
| 3     | 0.0406     | 0.8923    | Признаки переобучения |
| 4     | 0.0209     | **0.9146**| Новый максимум      |

**Инсайты**:
- Модель быстро сходится (пик F1 на 2 эпохе).
- Переобучение на 3 эпохе: Train Loss ↓, Val F1 ↓.
- Лучшая модель сохранена на 4 эпохе, несмотря на рост Train Loss.

---

 4. **Рекомендации для BERT**
1. **Оптимизация обучения**:
   - Внедрить **early stopping** при падении Val F1.
   - Тестировать **заморозку эмбеддингов** BERT + дообучение верхних слоёв.
2. **Улучшение качества**:
   - Калибровка порога классификации для баланса FP/FN.
   - Добавить **контекстные правила** (например, блокировка определённых шаблонов токсичности).
3. **Скорость инференса**:
   - Экспериментировать с **квантованием модели** или **DistilBERT**.
   - Увеличить батч-сайз при прогнозировании (сейчас 10.46 примеров/сек).

## Итоговые выводы

 1. **Сравнение всех моделей**
 
| Критерий          | LinearSVM         | LightGBM          | BERT              |
|--------------------|-------------------|-------------------|-------------------|
| **Скорость**       | 🟢 Лучшая (8.4 сек)| 🔴 Самая медленная | 🟡 Умеренная      |
| **Toxic F1**       | 0.7783            | 0.7645            | **0.9146**        |
| **Интерпретируемость** | 🟢 Высокая     | 🟢 Средняя        | 🔴 Низкая         |
| **Масштабируемость** | 🟢 Для больших данных | 🟡 Ограничено | 🔴 Требует GPU   |

---

 2. **Рекомендации для проекта**
- **Выбор модели**:
  - **BERT** — для production, если критично качество и есть GPU-ресурсы.
  - **LinearSVM** — для MVP или систем с ограниченными мощностями.
- **Доработки**:
  - Для BERT: оптимизировать скорость инференса (квантование, батчинг).
  - Для LinearSVM: повысить Recall через oversampling или ансамбли.
- **Риски**:
  - **BERT**: Высокие затраты на обслуживание.
  - **LinearSVM**: Риск пропуска токсичного контента (FN=817).

---

 3. **Стратегия внедрения**
1. **Пилотная фаза**:
   - Запустить BERT в тестовом режиме с мониторингом FP/FN.
   - Сравнить нагрузку на инфраструктуру с текущими моделями.
2. **Оптимизация**:
   - Реализовать кэширование предсказаний для частых запросов.
   - Добавить фильтр-правила для очевидных случаев (маты, угрозы).
3. **Долгосрочный план**:
   - Переход на **DistilBERT** или **TinyBERT** для баланса скорости/качества.
   - Внедрение активного обучения для улучшения модели на реальных данных.

---

**Финальный вердикт**:  
BERT — безусловный лидер по качеству, но требует значительных ресурсов.  
Для стартапов или проектов с ограничениями LinearSVM остается жизнеспособной альтернативой.  